In [ ]:
import os
import torch
import pandas as pd
import numpy as np
import torchvision
import torch.nn as nn
from torch.utils.data import Dataset, random_split, DataLoader
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torchvision.models as models
from PIL import Image
# from collections import OrderedDict
%matplotlib inline

import torchvision.transforms as transforms

# 계층적인 폴더 구조를 갖고 있는 데이터셋을 불러올때 사용 : 폴더 이름 = 클래스 명
from torchvision.datasets import ImageFolder

from torchvision.models import resnet152
import torch.optim as optim

from tqdm.auto import tqdm

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device =='cuda':
    torch.cuda.manual_seed_all(777)
    
batch_size = 64
num_epochs = 200
learning_rate = 0.01

In [6]:
# another
dataset = ImageFolder('../data/Images')

test_pct = 0.2
test_size = int(len(dataset)*test_pct)
dataset_size = len(dataset) - test_size

val_pct = 0.1
val_size = int(dataset_size*val_pct)
train_size = dataset_size - val_size

train, val, test = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

# custom Dataset
class DogData(Dataset) :
    def __init__(self, ds, transform = None) :
        self.ds = ds
        self.transform = transform
    
    def __len__(self) :
        return len(self.ds)
    
    def __getitem__(self, idx) :
        img, label = self.ds[idx]
        if self.transform :
            img = self.transform(img)
            return img, label

        
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225]) 
                                      ])

val_transforms = transforms.Compose([
                                     transforms.Resize(255), 
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([
                                     transforms.Resize(255), 
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])


train = DogData(train, train_transforms)
val = DogData(val, val_transforms)
test = DogData(test, test_transforms)


num_workers = 0

trainLoader = torch.utils.data.DataLoader(train, batch_size=batch_size, 
                                              num_workers=num_workers, shuffle=True)
valLoader = torch.utils.data.DataLoader(val, batch_size=batch_size, 
                                            num_workers=num_workers, shuffle=False)
testLoader = torch.utils.data.DataLoader(test, batch_size=batch_size,
                                             num_workers=num_workers, shuffle=False)

In [8]:
resnet = resnet152(pretrained=True)

In [9]:
class Resnet(nn.Module):
    
    def __init__(self, resnet, n_classes, freeze=True):
        super().__init__()
        self.resnet = resnet
        if freeze:
            for param in resnet.parameters():
                param.requires_grad = False
        n_inputs = self.resnet.fc.out_features # 1000
        # 학습시킬 파라미터
        self.fc1 = nn.Linear(n_inputs, 1024)
        self.fc2 = nn.Linear(1024, n_classes)
        
    def forward(self, x):
        x = self.resnet(x)
        x = torch.relu(self.fc1(x))
        return torch.relu(self.fc2(x))

In [13]:
# learning rate를 단계적으로 줄여주는 방법
# epoch 100 -> lr/10, 150 -> lr/10
def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    if epoch >= 100:
        lr /= 10
    if epoch >= 150:
        lr /= 10
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

### Train & validation

In [14]:
# loss
criterion = nn.CrossEntropyLoss()
# backpropagation method
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0002)
# hyper-parameters
num_batches = len(trainLoader)

In [ ]:
trn_loss_list = []
val_loss_list = []
for epoch in tqdm(range(num_epochs)):
    adjust_learning_rate(optimizer,epoch)
    trn_loss = 0.0
    for i, data in enumerate(trainLoader):
        x, label = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        model_output = model(x)
        loss = criterion(model_output, label)
        loss.backward()
        optimizer.step()
        
        trn_loss += loss.item()
        
        if (i+1) % (num_batches // 2) == 0: # every 100 mini-batches
            with torch.no_grad(): # very very very very important!!!
                val_loss = 0.0
                corr_num = 0
                total_num = 0
                for j, val in enumerate(valLoader):
                    val_x, val_label = val[0].to(device), val[1].to(device)
                    val_output = model(val_x)
                    v_loss = criterion(val_output, val_label)
                    val_loss += v_loss
                    
                    model_label = val_output.argmax(dim=1)
                    corr = torch.eq(val_label, model_label).sum()
                    corr_num += corr.item()
                    total_num += val_label.size(0)
            
                print(f"epoch: {epoch+1:03d}/{num_epochs} | "
                      f"step: {i+1:03d}/{num_batches} | "
                      f"trn loss: {trn_loss/100:08.4f} "
                      f"| val loss: {val_loss/len(valLoader):08.4f} "
                      f"| acc: {(corr_num/total_num)*100:.2f}")           
            
            trn_loss_list.append(trn_loss/100)
            val_loss_list.append(val_loss/len(valLoader))
            trn_loss = 0.0

print("training finished!")

epoch: 001/200 | step: 116/232 | trn loss: 003.4952 | val loss: 001.7386 | acc: 64.70
epoch: 001/200 | step: 232/232 | trn loss: 002.3490 | val loss: 001.4075 | acc: 70.78
epoch: 002/200 | step: 116/232 | trn loss: 002.1740 | val loss: 001.2127 | acc: 74.18
epoch: 002/200 | step: 232/232 | trn loss: 002.0295 | val loss: 001.2013 | acc: 73.88
epoch: 003/200 | step: 116/232 | trn loss: 001.8850 | val loss: 001.0433 | acc: 76.37
epoch: 003/200 | step: 232/232 | trn loss: 001.8532 | val loss: 001.0472 | acc: 75.88


In [ ]:
# save
PATH = "model.pt"
torch.save(model.state_dict(), PATH)

print("model saved!")

In [ ]:
# test acc
with torch.no_grad():
    corr_num = 0
    total_num = 0
    for j, val in enumerate(testLoader):
        val_x, val_label = val
        val_x = val_x.to(device)
        val_label =val_label.to(device)
        val_output = model(val_x)
        model_label = val_output.argmax(dim=1)
        corr = val_label[val_label == model_label].size(0)
        corr_num += corr
        total_num += val_label.size(0)

print("test_acc: {:.2f}".format(corr_num / total_num * 100))

In [ ]:
plt.figure(figsize = (15,10))
plt.plot(trn_loss_list, label="train_loss")
plt.legend()
plt.grid('on')
plt.show()